# GTEx v8 - Download and Basic library preparation

Downloads the files.  Shows data and some plot

In [ ]:
# Graphing library imports

import os, sys, glob

import inspect, urllib

import pandas
import scipy
import numpy as np
from pprint import pprint

import time, datetime

import seaborn as sns

import plotly.express as px

# from chart_studio import plotly as py
import plotly.figure_factory as ff
import plotly.graph_objects as go


## Download GTEX data locally

In [ ]:
## Comments below for nbdev_build_lib to export to fleet_gene/gtex.py
# default_exp gtex
# export

from logzero import logger
import pathlib
import pickle
import os
from tqdm import tqdm
import requests


# GTEx data from https://gtexportal.org/home/datasets
# Gene transcripts per million data

GTEX_URL =  "https://storage.googleapis.com/gtex_analysis_v8"

GTEXV8_TPM = os.path.join(GTEX_URL, "rna_seq_data", "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz")
GTEXV8_TPM_MED = os.path.join(GTEX_URL, "rna_seq_data", "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz")
GTEX_PHENO_DS = os.path.join(GTEX_URL, "annotations", "GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt")
GTEX_PHENO_DD = os.path.join(GTEX_URL, "annotations", "GTEx_Analysis_v8_Annotations_SubjectPhenotypesDD.xlsx")
GTEX_SAMPLE_DS = os.path.join(GTEX_URL, "annotations", "GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt")
GTEX_SAMPLE_DD = os.path.join(GTEX_URL, "annotations", "GTEx_Analysis_v8_Annotations_SampleAttributesDD.xlsx")

script_dir = pathlib.Path().resolve()
cache_dir = os.path.join(script_dir, "data")

if os.path.exists(cache_dir):
    logger.info(f"Found {cache_dir}")
else:
    os.mkdir(cache_dir)

for url in [GTEXV8_TPM_MED, GTEXV8_TPM, GTEX_PHENO_DS, GTEX_PHENO_DD, GTEX_SAMPLE_DS, GTEX_SAMPLE_DD]:
    dest = os.path.join(cache_dir, os.path.basename(url))
    if os.path.exists(dest):
        logger.info(f"found existing: {dest}")
    else:
        logger.info(f"Downloading {dest}")
        # Open a handle onto the GTEx expression data
        response = requests.get(url, stream=True)

        with open(dest, "wb") as fh:
            for data in tqdm(response.iter_content()):
                fh.write(data)
        logger.info(f"Completed {dest}")

[I 210125 12:38:30 <ipython-input-26-a23e22683882>:29] Found /home/dustin/fleet_gene/data
[I 210125 12:38:30 <ipython-input-26-a23e22683882>:36] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz
[I 210125 12:38:30 <ipython-input-26-a23e22683882>:36] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz
[I 210125 12:38:30 <ipython-input-26-a23e22683882>:36] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt
[I 210125 12:38:30 <ipython-input-26-a23e22683882>:36] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDD.xlsx
[I 210125 12:38:30 <ipython-input-26-a23e22683882>:36] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt
[I 210125 12:38:30 <ipython-input-26-a23e22683882>:36] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations

In [ ]:
# Some Details about the GTEX data

# Detailed data file:
gtex_tpm_fn = os.path.join(cache_dir, os.path.basename(GTEXV8_TPM))

# Just looking at median values by tissue (SMTSD) greatly reduces file size
gtex_tpm_med_fn = os.path.join(cache_dir, os.path.basename(GTEXV8_TPM_MED))


# Sample annotation data

# Subject Phenotype data
# Age, sex and Hardy Scale death circumstances
gtex_pheno_fn =  os.path.join(cache_dir, os.path.basename(GTEX_PHENO_DS))

# Main sample data of interest is:
# 'SMTS': 'Tissue Type, area from which the tissue sample was taken.  This is a parent value to SMTSD.'
# 'SMTSD': 'SMTS Detailed'
gtex_attr_fn = os.path.join(cache_dir, os.path.basename(GTEX_SAMPLE_DS))
gtex_attr_desc_fn = os.path.join(cache_dir, os.path.basename(GTEX_SAMPLE_DD))

# Load tissue specific detatils
gtex_attr = pandas.read_csv(gtex_attr_fn, sep='\t')
tissue_types = sorted(gtex_attr['SMTS'].unique())
tissue_dict = dict()
for tissue in tissue_types:
    tissue_dict[tissue] = dict()
    tissue_dict[tissue]['samples'] = gtex_attr[gtex_attr['SMTS'] == tissue]['SAMPID'].to_list()
    tissue_dict[tissue]['subtypes'] = dict()
    for subtype in gtex_attr[gtex_attr['SMTS'] == tissue]['SMTSD'].unique():
        # Get all samples by subtype
        tissue_dict[tissue]['subtypes'][subtype] = gtex_attr[gtex_attr['SMTSD'] == subtype]['SAMPID'].to_list()
    

    sorted(gtex_attr[gtex_attr['SMTS'] == tissue]['SMTSD'].unique())
tissue_subtypes = sorted(gtex_attr['SMTSD'].unique())

# log number of tissue samples
logger.info('%s separate tissue types - total samples %s', len(tissue_dict), len(gtex_attr['SAMPID']))
for tissue in sorted(tissue_dict.keys()):
    logger.info('%s (n=%s)', tissue, len(tissue_dict[tissue]['samples']))
    if len(tissue_dict[tissue]['subtypes']) > 1:
        for subtype in sorted(tissue_dict[tissue]['subtypes'].keys()):
            logger.info('\t%s (n=%s)', subtype, len(tissue_dict[tissue]['subtypes'][subtype]))


[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:39] 31 separate tissue types - total samples 22951
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:41] Adipose Tissue (n=1327)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:44] 	Adipose - Subcutaneous (n=763)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:44] 	Adipose - Visceral (Omentum) (n=564)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:41] Adrenal Gland (n=275)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:41] Bladder (n=21)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:41] Blood (n=3480)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:44] 	Cells - EBV-transformed lymphocytes (n=192)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:44] 	Whole Blood (n=3288)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:41] Blood Vessel (n=1473)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:44] 	Artery - Aorta (n=450)
[I 210125 10:35:01 <ipython-input-3-50b7829108bd>:44] 	Artery - Coronary (n=253)
[I 210125 10:35:01 

In [ ]:
# Download the files to a local hard drive

if os.path.exists(cache_dir):
    logger.info(f"Found {cache_dir}")
else:
    os.mkdir(cache_dir)

for url in [GTEXV8_TPM_MED, GTEXV8_TPM, GTEX_PHENO_DS, GTEX_PHENO_DD, GTEX_SAMPLE_DS, GTEX_SAMPLE_DD]:
    dest = os.path.join(cache_dir, os.path.basename(url))
    if os.path.exists(dest):
        logger.info(f"found existing: {dest}")
    else:
        logger.info(f"Downloading {dest}")
        response = requests.get(url, stream=True)

        with open(dest, "wb") as fh:
            for data in tqdm(response.iter_content()):
                fh.write(data)
        logger.info(f"Completed {dest}")

[I 210125 12:40:00 <ipython-input-27-16ead20aa256>:4] Found /home/dustin/fleet_gene/data
[I 210125 12:40:00 <ipython-input-27-16ead20aa256>:11] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz
[I 210125 12:40:00 <ipython-input-27-16ead20aa256>:11] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz
[I 210125 12:40:00 <ipython-input-27-16ead20aa256>:11] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt
[I 210125 12:40:00 <ipython-input-27-16ead20aa256>:11] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDD.xlsx
[I 210125 12:40:00 <ipython-input-27-16ead20aa256>:11] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt
[I 210125 12:40:00 <ipython-input-27-16ead20aa256>:11] found existing: /home/dustin/fleet_gene/data/GTEx_Analysis_v8_Annotations_

In [ ]:
# Some table loading helper functions

# The full set of all all GTEx tpms is very large and can be hard to work with
# Some functions only load some of the data at a time.

# Genes of interest - some test genes 
TEST_GENES = [
    'TP53',
    'ERBB2',  # Herceptin target
    'EGFR',
    'AKT1',
    'KRAS',
    'PTEN',
    'APOE',
]

TUMOUR_MUT_GENES = [  # www.tumourportal.org - highly mutated
    'TP53',
    'PIK3CA',
    'PTEN',
    'KRAS',
    'APC',
    'MLL3',  # aka KMT2C
    'KMT2C',
    'FAT1',
    'MLL2',  # aka KMT2D
    'KMT2D',
    'ARID1A',
    'VHL',
    'PBRM1',
    'NF1',
    'EGFR',
    'ATM',
    'PIK3R1',
    'BRAF',
    'CDKN2A',
    'SETD2',
    'CREBBP',
    'FBXW7',
    'SPEN',
    'MTOR',
    'RB1',
    'SMARCA4',
    'NOTCH1',
]

TEST_GENES = sorted(set(TEST_GENES + TUMOUR_MUT_GENES))

# gtex_tpm file has genes as rows.
# First two columns are gene information
# Over 10000 columns of samples

#                Name Description  GTEX-1117F-0226-SM-5GZZ7  GTEX-111CU-1826-SM-5GZYN ...
#0  ENSG00000223972.4     DDX11L1                   0.10820                   0.11580
#1  ENSG00000227232.4      WASH7P                  21.40000                  11.03000


GTPM_SKIP = 2  #  skip two comment rows at the beginning of the file

# Just getting a few rows of this giant file
logger.info('Loading Row partial table ...')
gtpm_row_table = pandas.read_csv(gtex_tpm_fn, skiprows=2, sep='\t', usecols=range(4))
logger.info('Loading column partial table ...')
gtpm_column_table = pandas.read_csv(gtex_tpm_fn, skiprows=lambda x: x not in [2, 3], sep='\t')

def gtpm_row_by_genename(gene_name):
    sub_table = gtpm_row_table[gtpm_row_table['Description'] == gene_name]
    if sub_table.empty:
        logger.error("Missing '%s row in GTEX TPM table'", gene_name)
        return None
    return [GTPM_SKIP + 1 + i for i in sub_table.index][0]

def gtpm_keep_rows(gene_list=TEST_GENES):
    row_list = [gtpm_row_by_genename(gene) for gene in gene_list]
    return [GTPM_SKIP] + [gene for gene in row_list if gene]


def gtex_tpm_partial_table_load(gene_list, sample_list=None):
    start_time = time.time()
    if gene_list:
        keep_rows = gtpm_keep_rows(gene_list)
        skiprows_func = lambda x: x not in keep_rows
        logger.info('loading %s GTEX TPM rows', len(keep_rows))
    else:
        logger.info('loading all rows')
        skiprows_func = lambda x: x in [0, 1]
    
    if sample_list:
        logger.info('Limiting load to %s potential samples', len(sample_list))
        # Not all samples are in tables
        sample_list = [s for s in sample_list if s in gtpm_column_table.columns]
        logger.info('Limiting load to %s samples', len(sample_list))
        columns = ['Name', 'Description'] + sample_list
    else:
        sample_list = None
    
    gtpm_partial = pandas.read_csv(
        gtex_tpm_fn,
        skiprows=skiprows_func,  # Only selected gene rows
        sep='\t',
        header=0,
        usecols=sample_list,  # Number of samples limit
    )
    end_time = time.time()
    logger.info("Gtex TPM partial table took {} minutes".format((end_time - start_time)/60))
    return gtpm_partial


[I 210125 10:54:13 <ipython-input-4-b213538e8d2c>:61] Loading Row partial table ...
[I 210125 10:54:47 <ipython-input-4-b213538e8d2c>:63] Loading column partial table ...


### Cache sample and gene lists

Preload and save these lists so that we can load part of the GTEX tables easily

In [ ]:
print(gtpm_column_table.columns[:3])
assert gtpm_column_table.columns[0] == "Name"
assert gtpm_column_table.columns[1] == "Description"

# Remaining columns are GTEX samples that start with GTEX-
GTEX_SAMPLES = gtpm_column_table.columns[2:].tolist()
for sample in GTEX_SAMPLES:
    assert sample.startswith("GTEX-")

logger.info(f"GTEX data has {len(GTEX_SAMPLES)} samples")

GTEX_GENES_ENSG = gtpm_row_table["Name"].tolist()
GTEX_GENES_SYMBOL = gtpm_row_table["Description"].tolist()
print(GTEX_GENES_ENSG[:2])
print(GTEX_GENES_SYMBOL[:2])

pseudo = []
for ensg, symbol in zip(GTEX_GENES_ENSG, GTEX_GENES_SYMBOL):
    assert ensg.startswith("ENSG")
    assert "." in ensg
    version = ensg[ensg.index(".") + 1:]
    try:
        int(version)
    except ValueError as err:
        if version.endswith("_PAR_Y"):
            # PAR1, PAR2, are homologous sequences of nucleotides on the X and Y chromosomes
            logger.debug(f"Nonstandard ENSG in {symbol} {ensg} version: {version}")
            pseudo.append((ensg, symbol))
        else:
            logger.error(f"Nonstandard ENSG {ensg} version: {version}")
print(f"GTEX data on {len(GTEX_GENES_ENSG)} genes")

[I 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:10] GTEX data has 17382 samples
[D 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:27] Nonstandard ENSG in LL0YNC03-29C1.1 ENSG00000228572.7_PAR_Y version: 7_PAR_Y
[D 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:27] Nonstandard ENSG in PLCXD1 ENSG00000182378.13_PAR_Y version: 13_PAR_Y
[D 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:27] Nonstandard ENSG in GTPBP6 ENSG00000178605.13_PAR_Y version: 13_PAR_Y
[D 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:27] Nonstandard ENSG in LINC00685 ENSG00000226179.6_PAR_Y version: 6_PAR_Y
[D 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:27] Nonstandard ENSG in PPP2R3B ENSG00000167393.17_PAR_Y version: 17_PAR_Y
[D 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:27] Nonstandard ENSG in RP13-465B17.4 ENSG00000281849.3_PAR_Y version: 3_PAR_Y
[D 210125 12:17:52 <ipython-input-19-ce4b51075d0b>:27] Nonstandard ENSG in Metazoa_SRP ENSG00000275287.5_PAR_Y version: 5_PAR_Y
[D 210125 12:17:52 <ipyth

Index(['Name', 'Description', 'GTEX-1117F-0226-SM-5GZZ7'], dtype='object')
['ENSG00000223972.5', 'ENSG00000227232.5']
['DDX11L1', 'WASH7P']
GTEX data on 56200


In [ ]:
dup_symbols = [i for i in GTEX_GENES_SYMBOL if GTEX_GENES_SYMBOL.count(i) > 1]
num_ambiguous_recs = len(dup_symbols)
dup_symbols = sorted(set(dup_symbols))
print(f"GTEX has {len(dup_symbols)} duplicate 'Descriptions' involving {num_ambiguous_recs} Ensembl genes.")

# Verify there are only duplications in symbols and not Ensembl gene ids
assert len(GTEX_GENES_ENSG) == len(set(GTEX_GENES_ENSG))

# pickle these lists for quick reloading later
logger.info(f"Pickling gene lists to {cache_dir}")
pickle.dump(GTEX_GENES_ENSG, open(os.path.join(cache_dir, "GTEX_GENES_ENSG.pkl"), "wb"))
pickle.dump(GTEX_GENES_SYMBOL, open(os.path.join(cache_dir, "GTEX_GENES_SYMBOL.pkl"), "wb"))
pickle.dump(dup_symbols, open(os.path.join(cache_dir, "GTEX_GENES_SYMBOL_DUPLICATES.pkl"), "wb"))

[I 210125 12:41:48 <ipython-input-28-9f21507b2af4>:10] Pickling gene lists to /home/dustin/fleet_gene/data


GTEX has 199 duplicate 'Descriptions' involving 1807 Ensembl genes.


In [ ]:
# export

# just export the gene list constants for speed later
import os, pickle
GTEX_GENES_ENSG = pickle.load(open(os.path.join(cache_dir, "GTEX_GENES_ENSG.pkl"), "rb"))
GTEX_GENES_SYMBOL = pickle.load(open(os.path.join(cache_dir, "GTEX_GENES_SYMBOL.pkl"), "rb"))
GTEX_GENES_SYMBOL_DUPLICATES = pickle.load(open(os.path.join(cache_dir, "GTEX_GENES_SYMBOL_DUPLICATES.pkl"), "rb"))

NameError: name 'cache_dir' is not defined

In [ ]:
# Small table of median values per tissue only
logger.info(f"loading median GTEx tissue TPM values: {gtex_tpm_med_fn}")
gtpm_med = pandas.read_csv(gtex_tpm_med_fn, sep='\t', skiprows=2, low_memory=False)
# gtpm_breast = gtex_tpm_partial_table_load(gene_list=TEST_GENES, sample_list=tissue_dict['Breast']['samples'])

logger.info(f"loading GTEX TPMs for {len(TEST_GENES)} genes")
logger.info(f" GTEX GENES {TEST_GENES}")
gtpm = gtex_tpm_partial_table_load(gene_list=TEST_GENES,)

# Remove missing genes from the test
TEST_GENES = gtpm['Description'].to_list()

## Median TPM values and Tissue comparison

The smaller table of median TPM values can give us a good idea of the overall character of the different genes.

The spread includes extreme values, most of the tissues are similar.

Mean is ~ 16 - 17 TPM, but 50th percentile is 0 and 75th percentile is ~ 2.

This indicates that the TPM is not a normal distribution, but has a small number of extreme values.

In [ ]:

# Do something with the median values

rename_cols = dict()
for t in tissue_dict.keys():
    # pprint(tissue_dict[t]['subtypes'].keys())
    # pick a single representative tissue subtype
    subtype =  sorted(tissue_dict[t]['subtypes'].keys())[-1]
    if subtype != 'Cells - Leukemia cell line (CML)':
        rename_cols[subtype] = t
sample_columns = tuple(t
     for t in tissue_dict.keys()
    )

# gm = gtpm_med.drop('gene_id', axis=1).set_index('Description')
gm = gtpm_med.set_index('Description')

gm.rename(columns=rename_cols, inplace=True)
gm = gm[sorted(rename_cols.values())]
# Copy table so we can add and modify values
gm = gm.copy()

# Using gm table we can chacterize the genes a bit.
# the spread includes extreme values, most of the tissues are similar
# Mean is ~ 16 - 17 TPM, but 50th percentile is 0 and 75th percentile is ~ 2
pandas.set_option('max_columns', 60)
pandas.set_option('display.width', 120)
#print(gm.describe())
print(gm.loc[TEST_GENES].head())


In [ ]:

# Apply some known values
gm['mean'] = gm.apply(lambda row: row.mean(), axis=1)
gm['median'] = gm.apply(lambda row: row.median(), axis=1)
gm['std'] = gm.apply(lambda row: row.std(), axis=1)
#gm['mode'] = gm.apply(lambda row: row.mode(), axis=1)
gm['CoV'] = gm.apply(lambda row: np.inf if row['mean'] == 0 else row['std']/row['mean'], axis=1)
gm['sem'] = gm.apply(lambda row: row.sem(), axis=1)
gm.head()

# high expression # low deviation among tissues
gm_bench = gm[(gm['median'] >= 1)  # high expression
             & (gm['median'] - 3.5 * gm['std'] > 0)
            ]

#cross_tissue_genes = sorted(gm_bench['Description'].tolist())

print("Of {} genes, the top {} in stable cross-tissue expression are:".format(len(gm), len(gm_bench)))
#for gene in cross_tissue_genes:
#    print('\t{}'.format(gene))
gm_bench.sort_values(by='CoV')


In [ ]:
gm.sort_values(by='sem').head(30)

In [ ]:
gm.loc[TEST_GENES]

In [ ]:
# Transpose to get Gene Columns
gt = gtpm[gtpm.columns[1:]]
gt.set_index('Description', inplace=True)
org_cols = gt.columns

#gtt = gt.transpose()
#gtt.head()
gt.head()

In [ ]:
print(gt.loc['TP53'].describe())
print(gt.loc['ERBB2'].describe())
print(gt.loc['PTEN'].describe())

In [ ]:
#gt.loc['TP53'].value_counts(bins=100, sort=False)
#gt.loc['PTEN'].skew()
#gt.loc['PTEN'].kurtosis()
gt.loc['PTEN'].quantile(0.9)

gtpm.head()
#gt.T[TEST_GENES[:-1]].iplot(kind='box',)
#gt.T[['TP53', 'PTEN']].plot(kind='box',)

In [ ]:
import math

#gene = 'TP53'
gene = 'PTEN'
title = '{}: Samples {}'.format(gene, len(gt.loc[gene]))
#gtt[gene].iplot(kind='hist', title=title, yTitle='counts', xTitle='Expression (TPM)', bins=100)

#py.iplot(gt.loc[gene].to_list(), kind='hist', title=title, yTitle='counts', xTitle='Expression (TPM)', bins=100)
#gt.loc[gene].iplot(kind='hist', title=title, yTitle='counts', xTitle='Expression (TPM)', bins=100)
#gt.loc[gene].plot(kind='hist', title=title, bins=100)

glog2 = gt.loc[gene].apply(math.log2)
glog2.plot(kind='hist', title=title, bins=100)

px.histogram(gt.loc[gene])

In [ ]:

#print(ff.create_distplot.__doc__)
gene_list = ['TP53', 'PTEN', 'KRAS', 'ERBB2', 'CDKN2A']
gene_list = ['TP53', 'PTEN', 'KRAS',]
px.histogram([gt.loc[g] for g in gene_list], gene_list)

In [ ]:
# by tissue types
gene = 'TP53'
data_lists = []
labels = []
for tissue in tissue_dict.keys():
    samples = tissue_dict[tissue]['samples']
    # print(tissue)
    # print(len(samples))
    gt_samples = [s for s in samples if s in gt.columns]
    if len(gt_samples) > 20:
        dt = gt.loc[gene][gt_samples].dropna()
        if not dt.empty:
            data_lists.append(dt)
            labels.append('{}({}) ~ {}'.format(tissue, len(samples), round(dt.median(), 1)))
    else:
        print('Only {} samples for "{}"'.format(len(gt_samples), tissue))
# add all samples
dt = gt.loc[gene].dropna()
data_lists.append(dt)
labels.append('{} - all ({}) ~ {}'.format(gene, len(dt), round(dt.median(), 1)))

fig_tp53 = ff.create_distplot(data_lists, labels, show_hist=False, show_rug=False, show_curve=True)  # curve_type='normal')
fig_tp53.layout.update(title=gene)
fig_tp53.show()

In [ ]:
# Expected distributions are roughly log normal - not normal distributions.
dir(go)
gene='PTEN'
tpm_label = "TPM      {} skew: {} kurtosis: {}".format(gene,
                                                       round(gt.loc[gene].skew(), 2),
                                                       round(gt.loc[gene].kurtosis(), 2))
log2_label = "log2TPM {} skew: {} kurtosis: {}".format(gene,
                                                       round(gt.loc[gene].apply(np.log2).skew(), 2),
                                                       round(gt.loc[gene].apply(np.log2).kurtosis(), 2))

fig_log2vsNormal = ff.create_distplot([gt.loc[gene].apply(np.log2), gt.loc[gene]],
                         [log2_label, tpm_label],
                         bin_size=[0.1, 2],
                         curve_type='normal',)
fig_log2vsNormal.show()



# Data structure

Data like TPM is lognormal distributed. 
 * There cannot be a negative number of transcripts.
 * log transforms (like fold-change - log 2) are more normally disturbuted.
 
 More irregular curves, like TP53 show more log-normal distributions when clustered into relevent groups, like tissue types.
 
 Otherwise, something like sklearn.preprocessing.RobustScaler looks good for IQR scaling - robust to outliers.
 
 This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).
 
 
 ## Correlation
 
 Gene expressions are correlated / anti-correlated (mutual information/effect) in interaction pathways.  'Activation' of certiain pathways is of interest for cancer and drug effects measures.
 
 
 PCA with 'whitening' 

In [ ]:
# Normalize to log2RPKM - log normal data
# Data is now effectively in fold change units

# Use a minum value to prevent div by zero.
#safelog2 = lambda x: numpy.log2(x) if x else numpy.log2(sys.float_info.min)
safelog2 = lambda x: np.log2(x) if x else np.nan
gfc = gt.applymap(safelog2)

print(gfc.info())
print(gfc.head())

In [ ]:
# Try data reduction like PCA and LDA

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler

# Which way are we scaling? Rows or columns?
rs = RobustScaler()
scaler = rs.fit(gfc.T)
gfc_norm = pandas.DataFrame(scaler.transform(gfc.T), index=gfc.columns, columns=gfc.index)
gfc_norm.dropna(inplace=True)
print(gfc_norm.head())
gfc_norm.describe()


In [ ]:
# by tissue types
gene = 'PTEN'
data_lists = []
labels = []
for tissue in tissue_dict.keys():
    samples = tissue_dict[tissue]['samples']
    # print(tissue)
    # print(len(samples))
    gt_samples = [s for s in samples if s in gfc_norm.index]
    if len(gt_samples) > 20:
        dt = gfc_norm[gene][gt_samples].dropna()
        if not dt.empty:
            data_lists.append(dt)
            labels.append('{}({}) ~ {}'.format(tissue, len(samples), round(dt.median(), 1)))
    else:
        print('Only {} samples for "{}"'.format(len(gt_samples), tissue))
# add all samples
dt = gfc_norm[gene].dropna()
data_lists.append(dt)
labels.append('{} - all ({}) ~ {}'.format(gene, len(dt), round(dt.median(), 1)))

fig_log2tissue = ff.create_distplot(data_lists, labels, show_hist=False, show_rug=False, show_curve=True)  # curve_type='normal')
fig_log2tissue.layout.update(title=gene)
fig_log2tissue.show()

In [ ]:
# imitate https://stackabuse.com/implementing-lda-in-python-with-scikit-learn/
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA

import sklearn.decomposition
print('sklearn.decomposition')
pprint([f for f in dir(sklearn.decomposition) if str(f)[0] != '_'])

import sklearn.discriminant_analysis
print('sklearn.discriminant_analysis')
pprint([f for f in dir(sklearn.discriminant_analysis) if str(f)[0] != '_'])

print('sklearn.ensemble')
import sklearn.ensemble
pprint([f for f in dir(sklearn.ensemble) if str(f)[0] != '_'])

print('sklearn.metrics')
import sklearn.metrics
pprint([f for f in dir(sklearn.metrics) if str(f)[0] != '_'])
pprint(dir(sklearn.metrics))



In [ ]:
# PCA
# sparse PCA is probably better - interpret which genes are more important.

pca3 = PCA(n_components=3)  # Alt opt PCA(0.99) - enough components for 99% of variance
sparse_pca = sklearn.decomposition.SparsePCA(n_components=3)

sparse_comp = sparse_pca.fit_transform(gfc_norm)
components3 = pca3.fit_transform(gfc_norm)

print(pandas.DataFrame(components3).head())
pca3.explained_variance_ratio_

In [ ]:
# Find the most important genes in pc1

sparse_comp = pandas.DataFrame(sparse_pca.components_, columns=gfc_norm.columns)
print(sparse_comp)
pc1 = sparse_comp.loc[0]
#pc1_genes = [g.index for g in pc1 if g != 0]
print(pc1)
pc1 = pc1.apply(abs)
#print(pc1.sort_values.__doc__)
pc1.sort_values(ascending=False, inplace=True)
print(pc1)
pc1_gene_list =[gene for gene in pc1.index if pc1[gene] != 0]
print(pc1_gene_list)


In [ ]:
# Some playing around with mutiindex for more useful tables
gtm = pandas.read_csv(gtex_tpm_med_fn, sep='\t', skiprows=2, low_memory=False)

gtm.set_index(['Description', 'Name'], inplace=True)

#tissues = [(col.split(' - ')[0], col.replace(col.split(' - ')[0] + ' - ', '')) for col in gtm.columns]
tissues = [col.split(' - ')[0] for col in gtm.columns]
subtypes =  [col.replace(col.split(' - ')[0] + ' - ', '') for col in gtm.columns]

gtm.columns = [tissues, subtypes]
print(gtm.head())

print(gtm.loc['TP53', 'Brain'])

# Makes some reasonable subtables.  With droplevel seems to work ok.

In [ ]:
gtm.loc[:, 'Brain']
#gtm.loc[:, gtm.columns.get_level_values(1) == 'Cortex']

# Full size - slow run

In [ ]:
gtex_attr.head()

In [ ]:
print(f'Full list of patient samples is {gtex_attr["SMAFRZE"].unique()}')
# Use only the RNASEQ set
print(f'Using only RNASEQ drops {len(gtex_attr[gtex_attr["SMAFRZE"] != "RNASEQ"])} records')
rna_seq_patients = gtex_attr[gtex_attr["SMAFRZE"] == "RNASEQ"]["SAMPID"].tolist()

In [ ]:
# Choose a random sample of half the patients
import random

random.seed(42)
patient_list = rna_seq_patients
patient_list = random.sample(rna_seq_patients, int(len(rna_seq_patients) / 4))

# Leaves a reserve list for validating with later.
reserve_patients = [patient for patient in rna_seq_patients if patient not in patient_list]

print(f'Total of {len(patient_list)} patients')
print(f'Using {len(patient_list)} RNASEQ patients with a reserve of {len(reserve_patients)})')

In [ ]:
logger.info("Starting g8 load")
_g8 = pandas.read_csv(
        gtex_tpm_fn,
        skiprows=2,  # First 2 rows are comments of file size - other func here to select genes
        sep='\t',
        header=0,
        usecols=["Description"] + patient_list,  # Number of samples/patients limit
    )
logger.info("\tfinished g8 load")
# g8 = _g8.drop('gene_id', axis=1)


In [ ]:
g8 = _g8.set_index('Description')
g8.rename(columns=rename_cols, inplace=True)
#g8.head()
dir(g8)

In [ ]:
# Descriptions have duplicate genes like 'RGS5'
_g8.drop_duplicates(subset=['Description'], keep="first", inplace=True)

g8 = _g8.set_index("Description")
g8t = g8.T
pca3 = PCA(n_components=3)  # Alt opt PCA(0.99) - enough components for 99% of variance
sparse_pca = sklearn.decomposition.SparsePCA(n_components=3)

logger.info("starting sparse_pca")
sparse_comp = sparse_pca.fit_transform(g8t)
logger.info("starting pca")
components3 = pca3.fit_transform(g8t)

print(pandas.DataFrame(components3).head())
pca3.explained_variance_ratio_

In [ ]:
sparse_comp = pandas.DataFrame(sparse_pca.components_, columns=g8t.columns)
# print(sparse_comp)
pc1 = sparse_comp.loc[0]
#pc1_genes = [g.index for g in pc1 if g != 0]
print(pc1)
pc1a = pc1.apply(abs)
#print(pc1.sort_values.__doc__)
pc1a.sort_values(ascending=False, inplace=True)
print(pc1a)
pc1_gene_list =[gene for gene in pc1a.index if pc1a.loc[gene] != 0]
#print(pc1_gene_list)

In [ ]:
pc1_gene_list

In [ ]:
sig_genes = []
for gene in pc1a.index:
    if pc1a[gene] > 0:
        sig_genes.append(gene)
len(sig_genes)

pprint(sig_genes[:50])

In [ ]:
pheno_desc = pandas.read_excel(os.path.join(cache_dir, os.path.basename(GTEX_PHENO_DD)))
pheno_desc

In [ ]:
pheno = pandas.read_csv(os.path.join(cache_dir, os.path.basename(GTEX_PHENO_DS)), sep="\t")
pheno.head()

In [ ]:
gtex_attr[["SAMPID", "SMTS", "SMTSD"]].head()

In [ ]:
tissue = gtex_attr[["SAMPID", "SMTS", "SMTSD"]].copy()
tissue["SMTS"] = tissue["SMTS"].astype("category")
tissue["SMTSD"] = tissue["SMTSD"].astype("category")

In [ ]:
train_df = tissue.merge(g8t, left_on="SAMPID", right_index=True)

In [ ]:
from fastai.tabular.all import Categorify, FillMissing, TabularPandas, cont_cat_split

In [ ]:
cont,cat = cont_cat_split(train_df, max_card=500, dep_var=["SMTS", "SMTSD"])

In [ ]:
cont,cat = cont_cat_split(train_df, max_card=500, dep_var="SMTSD")

In [ ]:
train_ob = TabularPandas(train_df, procs=[FillMissing, Categorify], cat_names=cat, cont_names=cont, y_names="SMTSD")

In [ ]:
dir()

In [ ]:
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules], key=lambda x: x[1], reverse=True)

#%memit

In [ ]:
%memit